In [1]:
import os
import sys
spark_home = r"C:\spark\spark-4.0.0-bin-hadoop3"
sys.path.insert(0, spark_home + r"\python")
sys.path.insert(0, spark_home + r"\python\lib\py4j-0.10.9.9-src.zip")
os.environ["PYSPARK_PYTHON"] = "C:/Users/Raghava/AppData/Local/Programs/Python/Python310/python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = "C:/Users/Raghava/AppData/Local/Programs/Python/Python310/python.exe"


In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, when, round
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
prices_df = spark.read.csv("H:/leet_code_sql_50/sql_50_leetcode/sample_data/16_prices.csv", header=True, inferSchema=True)

In [5]:
unitssold_df = spark.read.csv("H:/leet_code_sql_50/sql_50_leetcode/sample_data/16_unitssold.csv", header=True, inferSchema=True)

In [6]:
prices_df.show()

+----------+----------+----------+-----+
|product_id|start_date|  end_date|price|
+----------+----------+----------+-----+
|         1|2019-02-17|2019-02-28|    5|
|         1|2019-03-01|2019-03-22|   20|
|         2|2019-02-01|2019-02-20|   15|
|         2|2019-02-21|2019-03-31|   30|
+----------+----------+----------+-----+



In [7]:
unitssold_df.show()

+----------+-------------+-----+
|product_id|purchase_date|units|
+----------+-------------+-----+
|         1|   2019-02-25|  100|
|         1|   2019-03-01|   15|
|         2|   2019-02-10|  200|
|         2|   2019-03-22|   30|
+----------+-------------+-----+



In [10]:
result_df = prices_df.alias('p')\
    .join(unitssold_df.alias('u'), (col("p.product_id") == col("u.product_id")) &
          ((col("u.purchase_date") >= col("p.start_date")) & (col("u.purchase_date") <= col("p.end_date")))
          , how = 'left')

In [11]:
result_df.show()

+----------+----------+----------+-----+----------+-------------+-----+
|product_id|start_date|  end_date|price|product_id|purchase_date|units|
+----------+----------+----------+-----+----------+-------------+-----+
|         1|2019-02-17|2019-02-28|    5|         1|   2019-02-25|  100|
|         1|2019-03-01|2019-03-22|   20|         1|   2019-03-01|   15|
|         2|2019-02-01|2019-02-20|   15|         2|   2019-02-10|  200|
|         2|2019-02-21|2019-03-31|   30|         2|   2019-03-22|   30|
+----------+----------+----------+-----+----------+-------------+-----+



In [23]:
aggregated_df = result_df.groupBy(col('p.product_id'))\
    .agg(when(
    (_sum(col('u.units')) == 0)  | (_sum(col('u.units')).isNull()) ,0  )\
         .otherwise(round(_sum(col('u.units')*col('p.price'))*1.0/_sum(col('u.units')),2)).alias("average_price"))

In [24]:
aggregated_df.show()

+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+

